In [ ]:
%reload_ext autoreload
%autoreload 2
import os
os.environ["THEANO_FLAGS"] = "device=gpu1"
#os.environ["CPATH"] = "/home/gridcl/mehdicherti/build/cuda/include/"
#os.environ["LIBRARY_PATH"] = "/home/gridcl/mehdicherti/build/cuda/lib64/"

import numpy as np
import cPickle as pickle
%matplotlib inline

import scipy.ndimage as nd
from lasagnekit.misc.draw_net import draw_to_notebook, draw_to_file
from lasagnekit.misc.plot_weights import grid_plot
from lasagnekit.misc.dream import build_dreamer
from lasagnekit.easy import LightweightModel, BatchOptimizer

from lasagne import layers, updates

from IPython.display import SVG

import theano
import theano.tensor as T

from skimage.io import imread
from skimage.transform import resize

from lasagnekit.easy import get_stat

import matplotlib.pyplot as plt

from scipy.ndimage import gaussian_filter
from lasagne.layers import InputLayer, DenseLayer, NonlinearityLayer
from lasagne.layers import Conv2DLayer as ConvLayer
from lasagne.layers import Pool2DLayer as PoolLayer
from lasagne.nonlinearities import softmax

import pprint



In [ ]:

import PIL

from cStringIO import StringIO
import PIL.Image
from IPython.display import clear_output, Image, display

def showarray(a, fmt='jpeg'):
    a = np.uint8(np.clip(a, 0, 255))
    f = StringIO()
    PIL.Image.fromarray(a).save(f, fmt)
    display(Image(data=f.getvalue()))
    
def preprocess(mv, img):
    return np.float32(np.rollaxis(img, 2)[::-1]) - mv
def deprocess(mv, img):
    return np.dstack((img + mv)[::-1])

In [ ]:
import vgg19, googlenet

model_name = "vgg"

if model_name == "vgg":
    build_model = vgg19.build_model
    model_filename = "/home/mcherti/work/data/zoo/vgg19.pkl"
elif model_name == "googlenet":
    build_model = googlenet.build_model
    model_filename = "/home/mcherti/work/data/zoo/blvc_googlenet.pkl"

In [ ]:
# build ggl net and load weights
net = build_model()
model_data = pickle.load(open(model_filename))
values = model_data['param values']
layers.set_all_param_values(net['prob'], values)

In [ ]:
pprint.pprint(net.keys())

In [ ]:
model = net
classes = np.array(model_data["synset words"])

if "mean value" in model_data:
    mean_value = (model_data["mean value"])
else:
    mean_value = np.array([104.0, 116.0, 122.0])

mean_value = mean_value[:, np.newaxis, np.newaxis]

In [ ]:
draw_to_file(layers.get_all_layers(model['prob']), "{0}.svg".format(model_name))

In [ ]:
SVG("{0}.svg".format(model_name))

##MultiDeepDream

In [ ]:
t_generated_image = T.tensor4()
t_input_images = T.tensor4()
def total_variation_loss(x):
    return (((x[:,:,:-1,:-1] - x[:,:,1:,:-1])**2 + (x[:,:,:-1,:-1] - x[:,:,:-1,1:])**2)**1.25).sum()

def gram_matrix(x):
    x = x.flatten(ndim=3)
    g = T.tensordot(x, x, axes=([2], [2]))
    return g

if model_name == "googlenet":
    end_layers = ["inception_3a/output", "inception_3b/output"]
    """
    end_layers = [#'conv1/7x7_s2', 
                  'conv2/3x3', 
                  'inception_3a/output',  
                  'inception_3b/output', 
                  'inception_4a/output', 
                  'inception_4b/output', 
                  'inception_4c/output']
    end_layers += [
        'inception_5b/3x3', 
        'inception_5b/3x3_reduce', 
        'inception_5b/pool', 
        'inception_5b/output', 
        'inception_5b/5x5', 
        'inception_5b/1x1', 
        'inception_5b/pool_proj'
    ]
    """
elif model_name == "vgg":
    end_layers = ['conv4_1', 'conv5_1']

L = 0
L_from_hidden = 0
t_hidden = []
all_hidden_outputs = []
for layer_name in end_layers:    
    o_input_images = layers.get_output(model[layer_name], t_input_images)
    all_hidden_outputs.append(o_input_images)
    o_generated_image = layers.get_output(model[layer_name], t_generated_image)
    #o_generated_image = layers.get_output(model[layer_name], t_generated_image).mean(axis=0)
    L += 0.5 * ((o_input_images - o_generated_image)**2).sum()

    #hidden = T.TensorType('float32', (True,) * o_generated_image.ndim )
    hidden = T.tensor4()
    t_hidden.append(hidden)
    
    if layer_name == "conv4_1d":
        representation = lambda x:x
        coef = 0.001
    else:
        representation = gram_matrix
        coef = 0.001
    
    L_from_hidden += 0.5 * coef * (( representation(hidden) - representation(o_generated_image))  ** 2).sum()

    
L_from_hidden += 0.1e-7*total_variation_loss(o_generated_image)
#L += -(layers.get_output(model["prob"], t_generated_image)**2).sum()

get_grad = theano.function([t_input_images, t_generated_image], theano.grad(L, t_generated_image))
get_grad_from_hidden = theano.function([t_generated_image] +  t_hidden,
                                       theano.grad(L_from_hidden, t_generated_image))
                                       
get_loss =  theano.function([t_input_images, t_generated_image], L)
get_loss_from_hidden = theano.function([t_generated_image] + t_hidden, L_from_hidden)
get_hidden = theano.function([t_input_images], all_hidden_outputs)


In [ ]:
h, w = 300, 300

In [ ]:
images = [
    #np.float32(PIL.Image.open('sample4.png')),
    np.float32(PIL.Image.open('canard.jpg')),
    np.float32(PIL.Image.open('aigle.jpg')),
]
#showarray(PIL.Image.open('sample2.png'))

generated_image = np.random.uniform(0, 255, size=images[0].shape)
#generated_image = resize(np.float32(PIL.Image.open('sample.png')), (h, w), preserve_range=True)

generated_image = preprocess(mean_value, generated_image).astype(np.float32)
generated_image = resize(generated_image, (3, h, w), preserve_range=True).astype(np.float32)
generated_image = generated_image[np.newaxis, :, :, :]

images = [preprocess(mean_value, image).astype(np.float32) for image in images]
images = [resize(image, (3, h, w), preserve_range=True).astype(np.float32) for image in images]

In [ ]:
initialize_with_mean = False

if initialize_with_mean is True:
    generated_image = np.zeros(images[0].shape)
    for im in images[0:1]:
        generated_image += im
    generated_image /= len(images)
    generated_image = generated_image.astype(np.float32)
    generated_image = generated_image[np.newaxis, :, :, :]

In [ ]:
input_images = np.array(images)

In [ ]:
#l = get_loss(input_images, generated_image)
#h = get_hidden(generated_image)
#get_grad_from_hidden(generated_image, h)
#for lbfgs
from scipy.optimize import fmin_l_bfgs_b
def eval_loss(x0):
    x0 = (x0.reshape((1, 3, h, w))).astype(np.float32)
    generated_image_[:] = x0
    return get_loss_from_hidden(x0, *hidden_layers).astype(np.float64)

def eval_grad(x0):
    x0 = (x0.reshape((1, 3, h, w))).astype(np.float32)
    g = get_grad_from_hidden(x0, *hidden_layers)
    generated_image_[:] = x0
    return g.flatten().astype(np.float64)

In [ ]:
alpha = 0.01
use_lbfgs = True
generated_image_ = generated_image.copy()
curve = []
l = None
frames = []
nb_times = 20

hidden_layers_orig = get_hidden(images)
#print(hidden_layers_orig[0][0].shape)
#print(get_hidden(generated_image_)[0][0].shape)

for hidden in range(nb_times):
    #generated_image_ = generated_image.copy()

    print("Frame {}".format(hidden))
    hidden_layers = get_hidden(generated_image_)
    #print(hidden_layers[0].shape)
    #break
    #transform hidden layers here
    A = float(hidden) / nb_times
    #A = 0.5
    #A=0.5
    for i, hidden_layer in enumerate(hidden_layers):
        
        #if i == 2:
        #    hidden_layers[i][0] = hidden_layers_orig[i][0]
        #else:
        hidden_layers[i][0] = (1-A)*hidden_layers_orig[i][0] + A*hidden_layers_orig[i][1]
    for i in range(10):
        if use_lbfgs is True:      
            x0, _, _ = fmin_l_bfgs_b(eval_loss, 
                                     generated_image_.flatten(), 
                                     fprime=eval_grad, maxfun=50)
            generated_image_[:] = (x0.reshape(generated_image_.shape).astype(np.float32))
        else:
            # just use gradient descent if use_lbfgs is False
            # get gradient w.r.t the input
            g = get_grad_from_hidden(generated_image_, *hidden_layers)
            generated_image_ -= alpha * g    

        l = get_loss_from_hidden(generated_image_, *hidden_layers)
        curve.append(l)
        if i % 1==0:
            print(l)
        out = deprocess(mean_value, generated_image_[0])
        out = np.uint8(np.clip(out, 0, 255))
        frames.append(out)
        if i % 10==0:
            fig = plt.figure(figsize=(10, 10))        
            plt.title("generated")
            plt.imshow(out)
            plt.axis('off')
            plt.tight_layout()
            plt.show()

In [ ]:
plt.xlabel("iteration")
plt.ylabel("activation value")
plt.plot(curve)

In [ ]:
import matplotlib.animation as animation

class SubplotAnimation(animation.TimedAnimation):
    def __init__(self):
        fig = plt.figure()
        ax = fig.add_subplot(1, 1, 1)
        
        self.img1 = ax.imshow(np.random.uniform(0, 1, (h, w)), 
                               vmin=0, vmax=255, 
                               interpolation='none')
        animation.TimedAnimation.__init__(self, fig, interval=1000, blit=True)

    def _draw_frame(self, framedata):
        i = framedata
        self.img1.set_data(  frames[i] )
        self._drawn_artists = [self.img1]

    def new_frame_seq(self):
        return iter(range(len(frames)))

    def _init_draw(self):
         pass
ani = SubplotAnimation()
ani.save('video.mp4', fps=20)
plt.show()